In [ ]:
# Install only missing packages for Docker environment
!pip install transformers==4.39.3 safetensors==0.4.2 && \
python -c "import torch; print('torch:', torch.__version__)" && \
python -c "import transformers; print('transformers:', transformers.__version__)" && \
python -c "import safetensors; print('safetensors:', safetensors.__version__)"

  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
✅ torch: 2.8.0+cu128
✅ transformers: 4.39.3
✅ safetensors: 0.4.2


In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade "torch>=2.6" torchvision torchaudio


Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import all required libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.quantization import quantize_dynamic
import torch.nn.utils.prune as prune
import copy
import os
import matplotlib.pyplot as plt
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import Dataset as HFDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import datetime
import warnings
warnings.filterwarnings('ignore')

# Device configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print(f"Using device: {DEVICE}")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [ ]:
# Quick test to verify PyTorch checkpoints load correctly
def test_checkpoints():
    print("Testing PyTorch checkpoint loading...")

    # Test DeBERTa checkpoint
    try:
        print("\n1. Testing DeBERTa checkpoint...")
        deberta_model = AutoModelForSequenceClassification.from_pretrained(
            "agentlans/deberta-v3-base-tweet-sentiment",
            num_labels=5,ignore_mismatched_sizes=True
        )
        ckpt = torch.load(
            "/storage/yagel/ADL/checkpoints/deberta_hp_tuning_study/best_model/best_model.pt",
            map_location="cpu"
        )
        deberta_model.load_state_dict(ckpt['model_state_dict'], strict=False)
        print("DeBERTa checkpoint loaded successfully!")
        print(f"   Model type: {type(deberta_model).__name__}")
        print(f"   Number of parameters: {sum(p.numel() for p in deberta_model.parameters()):,}")
        del deberta_model

    except Exception as e:
        print(f"DeBERTa checkpoint failed: {e}")

    # Test RoBERTa checkpoint
    try:
        print("\n2. Testing RoBERTa checkpoint...")
        roberta_model = AutoModelForSequenceClassification.from_pretrained(
            "cardiffnlp/twitter-roberta-base-sentiment-latest",
            num_labels=5,
            ignore_mismatched_sizes=True
        )
        ckpt = torch.load(
            "/storage/yagel/ADL/checkpoints/roberta_hp_tuning_study/best_model/best_model.pt",
            map_location="cpu"
        )
        roberta_model.load_state_dict(ckpt['model_state_dict'], strict=False)
        print("RoBERTa checkpoint loaded successfully!")
        print(f"   Model type: {type(roberta_model).__name__}")
        print(f"   Number of parameters: {sum(p.numel() for p in roberta_model.parameters()):,}")
        del roberta_model

    except Exception as e:
        print(f"RoBERTa checkpoint failed: {e}")

    print("\n" + "="*50)
    print("Checkpoint test completed!")

# Run the test
test_checkpoints()

Testing PyTorch checkpoint loading...

1. Testing DeBERTa checkpoint...


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at agentlans/deberta-v3-base-tweet-sentiment and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ DeBERTa checkpoint loaded successfully!
   Model type: DebertaV2ForSequenceClassification
   Number of parameters: 184,425,989

2. Testing RoBERTa checkpoint...


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpo

✅ RoBERTa checkpoint loaded successfully!
   Model type: RobertaForSequenceClassification
   Number of parameters: 124,649,477

Checkpoint test completed!


In [ ]:
# Test loading HuggingFace checkpoints for RoBERTa and DeBERTa

def test_hf_checkpoints():
    print("Testing HuggingFace checkpoints...")

    # Test DeBERTa HF checkpoint
    try:
        print("\n1. Testing DeBERTa HF checkpoint...")
        deberta_model = AutoModelForSequenceClassification.from_pretrained(
            "/storage/yagel/ADL/deberta_results_HF/deberta_trial_4/checkpoint-4950"
        )
        print("DeBERTa HF checkpoint loaded successfully!")
        print(f"   Model type: {type(deberta_model).__name__}")
        print(f"   Number of parameters: {sum(p.numel() for p in deberta_model.parameters()):,}")
        del deberta_model
    except Exception as e:
        print(f"DeBERTa HF checkpoint failed: {e}")

    # Test RoBERTa HF checkpoint
    try:
        print("\n2. Testing RoBERTa HF checkpoint...")
        roberta_model = AutoModelForSequenceClassification.from_pretrained(
            "/storage/yagel/ADL/roberta_results_HF/trial_7/checkpoint-18000"
        )
        print("RoBERTa HF checkpoint loaded successfully!")
        print(f"   Model type: {type(roberta_model).__name__}")
        print(f"   Number of parameters: {sum(p.numel() for p in roberta_model.parameters()):,}")
        del roberta_model
    except Exception as e:
        print(f"RoBERTa HF checkpoint failed: {e}")

    print("\n" + "="*50)
    print("HF Checkpoint test completed!")

# Run the test
test_hf_checkpoints()

Testing HuggingFace checkpoints...

1. Testing DeBERTa HF checkpoint...
✅ DeBERTa HF checkpoint loaded successfully!
   Model type: DebertaV2ForSequenceClassification
   Number of parameters: 184,425,989

2. Testing RoBERTa HF checkpoint...
✅ RoBERTa HF checkpoint loaded successfully!
   Model type: RobertaForSequenceClassification
   Number of parameters: 124,649,477

HF Checkpoint test completed!


In [ ]:
def load_models():
    models = {}

    # Model 1: HuggingFace checkpoint 1 - DeBERTa (from checkpoint directory)
    print("Loading HF Model 1 - DeBERTa...")
    models['hf_deberta'] = AutoModelForSequenceClassification.from_pretrained(
        "/storage/yagel/ADL/deberta_results_HF/deberta_trial_4/checkpoint-4950"
    ).to(DEVICE)

    # Model 2: HuggingFace checkpoint 2 - RoBERTa (from checkpoint directory)
    print("Loading HF Model 2 - RoBERTa...")
    models['hf_roberta'] = AutoModelForSequenceClassification.from_pretrained(
        "/storage/yagel/ADL/roberta_results_HF/trial_7/checkpoint-18000"
    ).to(DEVICE)

    # Model 3: PyTorch checkpoint 1 - DeBERTa (fine-tuned)
    print("Loading PT Model 1 - DeBERTa (fine-tuned)...")
    models['pt_deberta'] = AutoModelForSequenceClassification.from_pretrained(
        "agentlans/deberta-v3-base-tweet-sentiment",
        num_labels=5,
        ignore_mismatched_sizes=True
    )
    deberta_ckpt = torch.load("/storage/yagel/ADL/checkpoints/deberta_hp_tuning_study/best_model/best_model.pt", map_location=DEVICE)
    models['pt_deberta'].load_state_dict(deberta_ckpt['model_state_dict'], strict=False)
    models['pt_deberta'] = models['pt_deberta'].to(DEVICE)

    # Model 4: PyTorch checkpoint 2 - RoBERTa (fine-tuned)
    print("Loading PT Model 2 - RoBERTa (fine-tuned)...")
    models['pt_roberta'] = AutoModelForSequenceClassification.from_pretrained(
        "cardiffnlp/twitter-roberta-base-sentiment-latest",
        num_labels=5,
        ignore_mismatched_sizes=True
    )
    roberta_ckpt = torch.load("/storage/yagel/ADL/checkpoints/roberta_hp_tuning_study/best_model/best_model.pt", map_location=DEVICE)
    models['pt_roberta'].load_state_dict(roberta_ckpt['model_state_dict'], strict=False)
    models['pt_roberta'] = models['pt_roberta'].to(DEVICE)

    return models

# Load all models
models = load_models()
print(f"Loaded {len(models)} models: {list(models.keys())}")

Loading HF Model 1 - DeBERTa...


In [ ]:
pip install sentencepiece


In [ ]:
train_df = pd.read_csv("data/train_df.csv")
eval_df  = pd.read_csv("data/eval_df.csv")
test_df  = pd.read_csv("data/test_df.csv")

# ensure int labels, keep your columns
for df in (train_df, eval_df, test_df):
    df["label"] = pd.to_numeric(df["label"], errors="coerce").astype(int)
    assert df["label"].between(0, 4).all(), f"Labels out of range in {df.shape}: {set(df['label'])}"

train_df = train_df[["CleanTweet","label"]].reset_index(drop=True)
eval_df  = eval_df[["CleanTweet","label"]].reset_index(drop=True)
test_df  = test_df[["CleanTweet","label"]].reset_index(drop=True)

class TweetsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=256):
        self.texts = dataframe["CleanTweet"].tolist()
        self.labels = dataframe["label"].astype(int).tolist()  # ensure plain ints
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),       # squeeze batch dim only
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

tokenizer = AutoTokenizer.from_pretrained("agentlans/deberta-v3-base-tweet-sentiment", use_fast=False)

train_dataset = TweetsDataset(train_df, tokenizer)
val_dataset   = TweetsDataset(eval_df,  tokenizer)
test_dataset  = TweetsDataset(test_df,  tokenizer)

train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=5, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=5, shuffle=False)

print(f"Dataset sizes - Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")


In [ ]:
# Load model from checkpoint
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load base DeBERTa-v3 tweet sentiment model first, then load your trained weights
model = AutoModelForSequenceClassification.from_pretrained(
    'agentlans/deberta-v3-base-tweet-sentiment',  # CHANGE: Update to match your model architecture (e.g., 'bert-base-uncased', 'roberta-base')
    num_labels=5,              # CHANGE: Update number of labels to match your task
    ignore_mismatched_sizes=True
)

# Load your trained checkpoint weights
checkpoint_path = "checkpoints/deberta_hp_tuning_study/best_model/best_model.pt"  #manual code  CHANGE: Update path to your checkpoint file
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)

# Extract model weights from checkpoint metadata
if 'model_state_dict' in checkpoint:
    model_weights = checkpoint['model_state_dict']
    print(f"Loaded checkpoint metadata - Study: {checkpoint.get('study_name', 'N/A')}")
    print(f"Best trial: {checkpoint.get('study_best_trial', 'N/A')}, Best accuracy: {checkpoint.get('best_val_accuracy', 'N/A')}")
else:
    model_weights = checkpoint

model.load_state_dict(model_weights)
model.to(DEVICE)

print(f"Model loaded from: {checkpoint_path}")
print(f"Model loaded on {DEVICE}")
print(f"Model type: {type(model).__name__}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
test_df.head()

# Quantization

In [ ]:
# Function to measure model size in KB
def get_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / 1024  # KB
    # Clean up temporary file
    os.remove(filename)
    return size

# Function to evaluate accuracy
def evaluate_accuracy(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}  # Move everything to the device

            outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
            predictions = outputs.logits.argmax(dim=1)
            correct += (predictions == batch["labels"]).sum().item()
            total += batch["labels"].size(0)

    return correct / total

# --- Main test loop ---
def quantization_sweep(model, test_loader):
    results = []

    # Original FP32 model on GPU
    model.eval()
    fp32_size = get_model_size(model, "fp32.pth")
    fp32_acc = evaluate_accuracy(model, test_loader, DEVICE)
    results.append(("FP32 (baseline)", fp32_size, fp32_acc, 0.0))

    # INT8 Dynamic quantization (Linear layers only)
    # Move model to CPU for quantization
    model_cpu = model.cpu()
    int8_model = quantize_dynamic(model_cpu, {nn.Linear}, dtype=torch.qint8)
    int8_size = get_model_size(int8_model, "int8.pth")
    
    # Evaluate quantized model on CPU (quantized models can't run on GPU)
    int8_acc = evaluate_accuracy(int8_model, test_loader, torch.device('cpu'))
    results.append(("INT8 Linear", int8_size, int8_acc, 100*(1 - int8_size/fp32_size)))

    # Move original model back to GPU
    model.to(DEVICE)

    # Print results
    print(f"{'Format':<15} {'Size (KB)':<12} {'Accuracy':<10} {'% Size Saved':<12}")
    print("-" * 55)
    for name, size, acc, saving in results:
        print(f"{name:<15} {size:<12.1f} {acc:<10.4f} {saving:<12.1f}")

    return results

In [ ]:
#DeBERTA manual code (pt checkpoint)
model.to(DEVICE)
results = quantization_sweep(model, test_loader)
print("Quantization results:", results)

In [ ]:
#RoBERTA manual (pt checkpoint)
# Paths & backbone
roberta_backbone = "cardiffnlp/twitter-roberta-base-sentiment-latest"
roberta_ckpt     = "checkpoints/roberta_hp_tuning_study/best_model/best_model.pt"

# Build loader with the **matching tokenizer**
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_backbone, use_fast=True)
test_dataset = TweetsDataset(test_df, roberta_tokenizer, max_length=256)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Optional: sanity on label range for 5 classes
assert test_df["label"].between(0, 4).all(), "Labels must be in [0..4] for num_labels=5."
invalid_labels = test_df[~test_df["label"].between(0, 4)]
if not invalid_labels.empty:
    print("❌ Invalid labels found:\n", invalid_labels)
    raise ValueError("Test set contains invalid labels.")

# Load model + weights
roberta_model = AutoModelForSequenceClassification.from_pretrained(
    roberta_backbone, num_labels=5, ignore_mismatched_sizes=True
)
state = torch.load(roberta_ckpt, map_location=DEVICE)
roberta_model.load_state_dict(state["model_state_dict"])
roberta_model.eval()
roberta_model = roberta_model.to(DEVICE)

# Sanity check: token IDs fit vocab
batch = next(iter(test_loader))
max_id = int(batch["input_ids"].max())
vocab_sz = roberta_model.roberta.embeddings.word_embeddings.num_embeddings
print(f"Max token id = {max_id} | vocab size = {vocab_sz}")
assert max_id < vocab_sz, "Token ID exceeds model vocab size – tokenizer/model mismatch."
DEVICE = torch.device("cpu")
roberta_model.to(DEVICE)
results = quantization_sweep(roberta_model, test_loader)

print("RoBERTa (PT) Quantization results:", results)



In [ ]:
# ===============================
# DeBERTa: HF checkpoint + sweep
# ===============================
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# DeBERTa paths and backbone
deberta_backbone = "agentlans/deberta-v3-base-tweet-sentiment"  # Or your custom one if different

# Tokenizer and loader
deberta_tokenizer = AutoTokenizer.from_pretrained(deberta_backbone, use_fast=True)
test_dataset = TweetsDataset(test_df, deberta_tokenizer, max_length=256)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Sanity check: label range
assert test_df["label"].between(0, 4).all(), "Labels must be in [0..4] for num_labels=5."

# Load model
deberta_model = AutoModelForSequenceClassification.from_pretrained(
    "deberta_results_HF/deberta_trial_4/checkpoint-4950", num_labels=5
)


# Run sweep on CPU
DEVICE = torch.device("cpu")
deberta_model.to(DEVICE)
results = quantization_sweep(deberta_model, test_loader)
print("DeBERTa (HF) Quantization results:", results)


In [ ]:
# ===============================
# RoBERTa: HF CHECKPOINT + SWEEP
# ===============================
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Tokenizer from base model
roberta_backbone = "cardiffnlp/twitter-roberta-base-sentiment-latest"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_backbone, use_fast=True)

# Loader
test_dataset = TweetsDataset(test_df, roberta_tokenizer, max_length=256)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Sanity check
assert test_df["label"].between(0, 4).all(), "Labels must be in [0..4] for num_labels=5."

# ✅ Load model from local HF checkpoint
roberta_ckpt = "roberta_results_HF/trial_7/checkpoint-18000"
roberta_model = AutoModelForSequenceClassification.from_pretrained(
    roberta_ckpt,
    num_labels=5,
    local_files_only=True  # <<< this tells HF to treat the path as local
)

# Run sweep
DEVICE = torch.device("cpu")
roberta_model.to(DEVICE)
results = quantization_sweep(roberta_model, test_loader)

print("RoBERTa (HF checkpoint) Quantization results:", results)


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# Pruning

In [ ]:
import torch
import torch.nn.utils.prune as prune
import copy
import matplotlib.pyplot as plt
import pandas as pd

def pruning_tradeoff_sweep_cpu(model, test_loader, amounts=None):
    """
    CPU-only pruning sweep with results table and plot
    
    Args:
        model: Input model to prune
        test_loader: DataLoader for evaluation
        amounts: List of pruning amounts (default: 10%, 20%, 30%, 40%, 50%)
    
    Returns:
        results: List of dictionaries with pruning results
    """
    if amounts is None:
        amounts = [0.1, 0.2, 0.3, 0.4, 0.5]  # 10%, 20%, 30%, 40%, 50%
    
    print("🔬 Starting CPU Pruning Sweep")
    print("=" * 40)
    
    results = []
    
    for amount in amounts:
        print(f"\n📊 Testing pruning amount: {amount*100:.0f}%")
        
        # Create a copy of the model for this pruning amount
        model_to_prune = copy.deepcopy(model)
        
        # Get parameters to prune (all Linear layers)
        parameters_to_prune = []
        for name, module in model_to_prune.named_modules():
            if isinstance(module, torch.nn.Linear):
                parameters_to_prune.append((module, 'weight'))
        
        print(f"Found {len(parameters_to_prune)} Linear layers to prune")
        
        # Apply global unstructured pruning
        prune.global_unstructured(
            parameters_to_prune,
            pruning_method=prune.L1Unstructured,
            amount=amount
        )
        
        # Calculate sparsity
        total_params = 0
        zero_params = 0
        for name, module in model_to_prune.named_modules():
            if isinstance(module, torch.nn.Linear) and hasattr(module, 'weight_mask'):
                total_params += module.weight.numel()
                zero_params += (module.weight_mask == 0).sum().item()
        
        actual_sparsity = zero_params / total_params if total_params > 0 else 0
        print(f"Actual sparsity: {actual_sparsity:.3f} ({actual_sparsity*100:.1f}%)")
        
        # Evaluate accuracy on CPU
        acc = evaluate_accuracy_cpu(model_to_prune, test_loader)
        
        results.append({
            'pruning_amount': amount,
            'actual_sparsity': actual_sparsity,
            'accuracy': acc
        })
        
        print(f"Accuracy: {acc:.4f}")
        
        # Clean up
        del model_to_prune
    
    # Print final results table
    print("\n📈 PRUNING RESULTS SUMMARY")
    print("=" * 50)
    print("Pruning | Actual  | Accuracy")
    print("Amount  | Sparsity|         ")
    print("-" * 50)
    for result in results:
        print(f"{result['pruning_amount']*100:6.0f}% | {result['actual_sparsity']*100:6.1f}%  | {result['accuracy']:.4f}")
    
    # Create plot
    print("\n📊 Generating plot...")
    results_df = pd.DataFrame(results)
    
    plt.figure(figsize=(10, 6))
    plt.plot(results_df['actual_sparsity'] * 100, results_df['accuracy'], 'bo-', linewidth=2, markersize=8)
    plt.xlabel('Sparsity (%)', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.title('Model Pruning: Sparsity vs Accuracy Trade-off', fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.xticks(range(10, 60, 10))
    
    # Add accuracy values as annotations
    for i, row in results_df.iterrows():
        plt.annotate(f'{row["accuracy"]:.3f}', 
                    (row['actual_sparsity']*100, row['accuracy']), 
                    textcoords="offset points", xytext=(0,10), ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Pruning analysis complete!")
    
    return results

def evaluate_accuracy_cpu(model, test_loader):
    """
    CPU-only accuracy evaluation
    
    Args:
        model: Model to evaluate
        test_loader: DataLoader for evaluation
    
    Returns:
        accuracy: Float accuracy value
    """
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    return correct / total

In [ ]:
# ===============================
# RoBERTa: PT checkpoint + PRUNING (CPU)
# ===============================
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DEVICE = torch.device("cpu")

# Backbone + PT checkpoint path
roberta_backbone = "cardiffnlp/twitter-roberta-base-sentiment-latest"  # 3-class base head
roberta_pt_ckpt  =  "checkpoints/roberta_hp_tuning_study/best_model/best_model.pt"

# Tokenizer & loader
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_backbone, use_fast=True)
test_dataset = TweetsDataset(test_df, roberta_tokenizer, max_length=256)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load base arch w/ new 5-class head, then load your fine-tuned PT weights
roberta_model = AutoModelForSequenceClassification.from_pretrained(
    roberta_backbone, num_labels=5, ignore_mismatched_sizes=True
)
state = torch.load(roberta_pt_ckpt, map_location=DEVICE)
roberta_model.load_state_dict(state["model_state_dict"], strict=False)
roberta_model.to(DEVICE).eval()

# Run CPU pruning sweep
pruning_results_roberta_pt = pruning_tradeoff_sweep_cpu(
    roberta_model, test_loader, amounts=[0.1, 0.2, 0.3, 0.4, 0.5]
)
pruning_results_roberta_pt


In [ ]:
# ===============================
# RoBERTa: HF checkpoint dir + PRUNING (CPU)
# ===============================
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DEVICE = torch.device("cpu")

# Your HF checkpoint directory
roberta_hf_ckpt = "roberta_results_HF/trial_7/checkpoint-18000"

# Use tokenizer from backbone (checkpoint dir may not have tokenizer files)
roberta_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", use_fast=True)
test_dataset = TweetsDataset(test_df, roberta_tokenizer, max_length=256)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load model from local HF checkpoint dir
roberta_model_hf = AutoModelForSequenceClassification.from_pretrained(
    roberta_hf_ckpt, local_files_only=True
)
# (optional) enforce 5 labels if config differs
if roberta_model_hf.config.num_labels != 5:
    roberta_model_hf.classifier.out_proj = torch.nn.Linear(
        roberta_model_hf.classifier.out_proj.in_features, 5
    )
    roberta_model_hf.config.num_labels = 5

roberta_model_hf.to(DEVICE).eval()

# Run CPU pruning sweep
pruning_results_roberta_hf = pruning_tradeoff_sweep_cpu(
    roberta_model_hf, test_loader, amounts=[0.1, 0.2, 0.3, 0.4, 0.5]
)
pruning_results_roberta_hf


In [ ]:
# ===============================
# DeBERTa: HF checkpoint dir + PRUNING (CPU)
# ===============================
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DEVICE = torch.device("cpu")

# HF checkpoint directory you trained with Trainer.save_model / checkpoint-*
deberta_hf_ckpt = "deberta_results_HF/deberta_trial_4/checkpoint-4950"

# Use tokenizer from backbone (checkpoint dir often lacks tokenizer files)
deberta_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base", use_fast=True)
test_dataset = TweetsDataset(test_df, deberta_tokenizer, max_length=256)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load model weights from local HF checkpoint dir
deberta_model_hf = AutoModelForSequenceClassification.from_pretrained(
    deberta_hf_ckpt, local_files_only=True   # treat as local path
)
# (optional) enforce num_labels=5 if your config wasn’t saved with 5:
if deberta_model_hf.config.num_labels != 5:
    deberta_model_hf.classifier = torch.nn.Linear(deberta_model_hf.classifier.in_features, 5)
    deberta_model_hf.config.num_labels = 5

deberta_model_hf.to(DEVICE).eval()

# Run CPU pruning sweep
pruning_results_deberta_hf = pruning_tradeoff_sweep_cpu(
    deberta_model_hf, test_loader, amounts=[0.1, 0.2, 0.3, 0.4, 0.5]
)
pruning_results_deberta_hf


In [ ]:
# ===============================
# DeBERTa: PT checkpoint + PRUNING (CPU)
# ===============================
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DEVICE = torch.device("cpu")

# Backbone + PT checkpoint path
deberta_backbone = "agentlans/deberta-v3-base-tweet-sentiment"
deberta_pt_ckpt  = "checkpoints/deberta_hp_tuning_study/best_model/best_model.pt"

# Tokenizer & loader (use backbone tokenizer)
deberta_tokenizer = AutoTokenizer.from_pretrained(deberta_backbone, use_fast=True)
test_dataset = TweetsDataset(test_df, deberta_tokenizer, max_length=256)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load base arch then your fine-tuned PT weights
deberta_model = AutoModelForSequenceClassification.from_pretrained(
    deberta_backbone, num_labels=5, ignore_mismatched_sizes=True
)
state = torch.load(deberta_pt_ckpt, map_location=DEVICE)
deberta_model.load_state_dict(state["model_state_dict"], strict=False)
deberta_model.to(DEVICE).eval()

# Run CPU pruning sweep (10–50%)
pruning_results_deberta_pt = pruning_tradeoff_sweep_cpu(
    deberta_model, test_loader, amounts=[0.1, 0.2, 0.3, 0.4, 0.5]
)
pruning_results_deberta_pt


# Knowledge Distillation

In [ ]:
pip install --upgrade transformers accelerate

In [ ]:
pip install --upgrade transformers

In [ ]:
pip show transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset as HFDataset
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd

def perform_knowledge_distillation(teacher_model, teacher_name, train_df, eval_df, device, 
                                 temperature=2.0, alpha=0.5, num_epochs=5, batch_size=64):

    
    print(f"\nStarting knowledge distillation with teacher: {teacher_name}")
    
    # Convert pandas DataFrames to Hugging Face Datasets
    train_hf_dataset = HFDataset.from_pandas(train_df)
    eval_hf_dataset = HFDataset.from_pandas(eval_df)
    
    # Set up tokenizer (use student tokenizer for tokenization)
    student_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)
    
    # Tokenization function
    def tokenize_function(examples):
        return student_tokenizer(examples["CleanTweet"], truncation=True, padding="max_length", max_length=256)
    
    # Tokenize datasets
    tokenized_train = train_hf_dataset.map(tokenize_function, batched=True)
    tokenized_eval = eval_hf_dataset.map(tokenize_function, batched=True)
    
    # Compute metrics function
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return {
            'accuracy': accuracy_score(labels, predictions),
            'f1': f1_score(labels, predictions, average='weighted'),
            'precision': precision_score(labels, predictions, average='weighted'),
            'recall': recall_score(labels, predictions, average='weighted')
        }
    
    # Define DistillationTrainer
    class DistillationTrainer(Trainer):
        def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.5, **kwargs):
            super().__init__(*args, **kwargs)
            self.teacher = teacher_model
            if self.teacher is not None:
                self.teacher.eval()
            self.temperature = temperature
            self.alpha = alpha
        
        def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
            labels = inputs.pop("labels")
            outputs_student = model(**inputs)
            
            with torch.no_grad():
                outputs_teacher = self.teacher(**inputs)
            
            # Cross entropy loss
            loss_ce = F.cross_entropy(outputs_student.logits, labels)
            
            # KL divergence loss
            loss_kl = F.kl_div(
                F.log_softmax(outputs_student.logits / self.temperature, dim=-1),
                F.softmax(outputs_teacher.logits / self.temperature, dim=-1),
                reduction="batchmean"
            ) * (self.temperature ** 2)
            
            # Combined loss
            loss = self.alpha * loss_ce + (1 - self.alpha) * loss_kl
            
            return (loss, outputs_student) if return_outputs else loss
    
    # Create fresh student model
    student = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=5
    ).to(device)
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./distilled_model_{teacher_name}",
        evaluation_strategy="epoch",
        save_strategy="no",                      # Don't save any checkpoints
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=2e-5,
        weight_decay=0.01,
        logging_steps=500,
        report_to=[],                        # Don't report to wandb/tensorboard
    )
    
    # Create trainer
    trainer = DistillationTrainer(
        model=student,
        teacher_model=teacher_model.to(device),
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=student_tokenizer,
        temperature=temperature,
        alpha=alpha
    )
    
    # Train and evaluate
    print(f"Training student model with {teacher_name} teacher...")
    trainer.train()
    
    print(f"Evaluating distilled model...")
    eval_result = trainer.evaluate()
    eval_result['teacher_model'] = teacher_name
    
    # Print results
    print(f"Results for {teacher_name}:")
    for key, value in eval_result.items():
        if key.startswith('eval_'):
            print(f"  {key}: {value:.4f}")
    
    # Save results to files
    import json
    import os
    
    # Create results directory if it doesn't exist
    results_dir = "./distillation_results"
    os.makedirs(results_dir, exist_ok=True)
    
    # Save detailed results as JSON
    results_file = os.path.join(results_dir, f"{teacher_name}_results.json")
    with open(results_file, 'w') as f:
        # Convert numpy types to native Python types for JSON serialization
        json_results = {}
        for key, value in eval_result.items():
            if hasattr(value, 'item'):  # numpy types
                json_results[key] = value.item()
            else:
                json_results[key] = value
        json.dump(json_results, f, indent=2)
    
    # Save summary results as CSV
    summary_file = os.path.join(results_dir, f"{teacher_name}_summary.csv")
    import pandas as pd
    summary_df = pd.DataFrame([eval_result])
    summary_df.to_csv(summary_file, index=False)
    
    # Save key metrics as text file for easy reading
    metrics_file = os.path.join(results_dir, f"{teacher_name}_metrics.txt")
    with open(metrics_file, 'w') as f:
        f.write(f"Knowledge Distillation Results for {teacher_name}\n")
        f.write("=" * 50 + "\n\n")
        for key, value in eval_result.items():
            if key.startswith('eval_'):
                f.write(f"{key}: {value:.4f}\n")
        f.write(f"\nTeacher Model: {teacher_name}\n")
        f.write(f"Student Model: distilbert-base-uncased\n")
        f.write(f"Training Epochs: {num_epochs}\n")
        f.write(f"Batch Size: {batch_size}\n")
        f.write(f"Temperature: {temperature}\n")
        f.write(f"Alpha: {alpha}\n")
    
    print(f"Results saved:")
    print(f"  - JSON: {results_file}")
    print(f"  - CSV: {summary_file}")
    print(f"  - Text: {metrics_file}")
    
    return eval_result

print("Knowledge distillation function loaded successfully!")

In [ ]:
# ===============================
# Distillation: DeBERTa HF → DistilBERT Student
# ===============================

# Load your DeBERTa HF checkpoint
teacher_path = "deberta_results_HF/deberta_trial_4/checkpoint-4950"
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    teacher_path,
    num_labels=5,
    local_files_only=True
).to(DEVICE).eval()

print("🔥 Distilling from DeBERTa HF (local checkpoint)")

# Perform distillation using the function
result_deberta_hf = perform_knowledge_distillation(
    teacher_model=teacher_model,
    teacher_name="DeBERTa_HF",
    train_df=train_df,
    eval_df=eval_df,
    device=DEVICE,
    num_epochs=5,
    batch_size=32
)

print(f"✅ Completed distillation with DeBERTa HF model")

In [ ]:
print("🔥 Distilling from: RoBERTa HF")
# Load your custom trained RoBERTa model
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment-latest", 
    num_labels=5, 
    ignore_mismatched_sizes=True
)
ckpt = torch.load("checkpoints/roberta_hp_tuning_study/best_model/best_model.pt", map_location=DEVICE)
teacher_model.load_state_dict(ckpt["model_state_dict"])
teacher_model = teacher_model.to(DEVICE)

# Perform distillation using the function
result_deberta = perform_knowledge_distillation(
    teacher_model=teacher_model,
    teacher_name="RoBERTa_PT",
    train_df=train_df,
    eval_df=eval_df,
    device=DEVICE,
    num_epochs=5,
    batch_size=64  
)

print(f"✅ Completed distillation with DeBERTa PT model")


In [ ]:
print("🔥 Distilling from: DeBERTa HF")
# Load your custom trained DeBERTa model
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    "agentlans/deberta-v3-base-tweet-sentiment", 
    num_labels=5, 
    ignore_mismatched_sizes=True
)
ckpt = torch.load("checkpoints/deberta_hp_tuning_study/best_model/best_model.pt", map_location=DEVICE)
teacher_model.load_state_dict(ckpt["model_state_dict"])
teacher_model = teacher_model.to(DEVICE)

# Perform distillation using the function
result_deberta = perform_knowledge_distillation(
    teacher_model=teacher_model,
    teacher_name="DeBERTa_PT",
    train_df=train_df,
    eval_df=eval_df,
    device=DEVICE,
    num_epochs=5,
    batch_size=32  # Using 32 like your original code
)

print(f"✅ Completed distillation with DeBERTa PT model")

In [ ]:
# Load your custom trained RoBERTa model from HF checkpoint
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    "roberta_results_HF/trial_7/checkpoint-18000",
    num_labels=5,
    local_files_only=True
).to(DEVICE)

print("🔥 Distilling from: RoBERTa HF")

# Perform distillation using the function
result_roberta_hf = perform_knowledge_distillation(
    teacher_model=teacher_model,
    teacher_name="RoBERTa_HF",
    train_df=train_df,
    eval_df=eval_df,
    device=DEVICE,
    num_epochs=5,
    batch_size=32
)

print(f"✅ Completed distillation with RoBERTa HF model")